## Overview

In this assignment, the student will learn how to accomplish the following:

1. Generate batches for skip-gram models
2. Implement two loss functions to train static word embeddings
3. Tune the parameters and hyperparameter for word embeddings
4. Apply the learned word embeddings to the word analogy task
5. Perform comparative and extrinsic evaluation (through the word analogy task) of multiple models, generate hypotheses based on their own results, and design subsequent scientific experiments to test their hypoetheses.

## 1. Dataset and Libraries

Much like the previous assignments, we start with obtaining the dataset. Please make sure you understand what is happening here, since future assignments will simply point to a corpus, and leave the rest to you as part of the work.

Below, you see the `wget` command being used. It will load the `text8` corpus, which is about 100 MB of cleaned text from Wikipedia. The file is 100,000,000 characters long and contains 17,005,207 words including 253,854 unique words and 71,290 unique frequent words.

In [ ]:
!wget http://mattmahoney.net/dc/text8.zip
!unzip text8.zip
!rm text8.zip

--2024-04-12 21:54:35--  http://mattmahoney.net/dc/text8.zip
Resolving mattmahoney.net (mattmahoney.net)... 34.198.1.81
Connecting to mattmahoney.net (mattmahoney.net)|34.198.1.81|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 31344016 (30M) [application/zip]
Saving to: ‘text8.zip’

text8.zip           100%[===================>]  29.89M  25.9MB/s    in 1.2s    

2024-04-12 21:54:37 (25.9 MB/s) - ‘text8.zip’ saved [31344016/31344016]

Archive:  text8.zip
  inflating: text8                   


Next, we import the required libraries and set up the random seeds.

The `typing` imports are needed for data type annotations, so if your code requires adding more data types to the line `from typing import ...`, that is completely acceptable.

In [ ]:
import collections
import json

import numpy as np
from scipy.spatial import distance

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import random
import math

from tqdm import tqdm

import os
import pickle

from typing import List, Tuple, Dict, Union

# Seeds for repeatable experiements. DO NOT CHANGE.
np.random.seed(42)
torch.manual_seed(42)

## 2. Data Preparation for Word2Vec

To create word vectors through training, you must extract training examples from the provided data. Your task involves developing a method that generates training examples in groups (i.e., *batches*). Specifically, for the skip-gram model, you will move a window across the data and extract training examples from the words within that window. This is a common technique you will see often in NLP and computer vision, called the *sliding window* approach.

For example, consider this text with a batch size of 8 and window size of 5:

> The quick brown fox jumps over the lazy dog.

The first window is

- <font color = red>[The quick brown fox jumps]</font> over the lazy dog.

We treat "brown" as the target word here, appearing in the context of four context words: "The", "quick", "fox", and "jumps". The training examples thus generated are

1. (brown, The)
2. (brown, quick)
3. (brown, fox)
4. (brown, jumps)

Now, we slide by one word to the next window:

- The <font color = red>[quick brown fox jumps over]</font> the lazy dog.

This gives us the training examples

5. (fox, quick)
6. (fox, brown)
7. (fox, jumps)
8. (fox, over)

At this point, we have 8 training instances, which is the batch size. So, even though we can continue sliding the window further for more instances, those instances will not be a part of this batch. Thus, we stop generating more instances now, and return the batch data.

Here are two helper methods provided already to assist you with building the dataset (this is similar to the initial steps in your previous assignments):

In [ ]:
def file_to_tokens(filename: str) -> list[str]:
    """
    This function reads a specified text file, reads its content, and splits the content into tokens based on spaces.
    Each token is converted to lowercase before being added to the returned list of tokens.

    Parameters:
    - filename (str): The path to the text file to be read and tokenized.

    Returns:
    - List[str]: A list of tokens (words), all converted to lowercase, extracted from the file.
    """
    with open(filename) as file:
        text = file.read()
        tokens = [token.lower() for token in text.strip().split(" ")]
    return tokens


def build_dataset(words: List[str], vocab_size: int) -> Tuple[List[int], List[Tuple[str, int]], Dict[str, int], Dict[int, str]]:
    """
    This function processes a list of words into a dataset, generating mappings between tokens and numeric identifiers,
    while also calculating the frequency of each word. It creates a vocabulary of the most common words up to the
    specified maximum size, assigning a unique identifier to each. Other words are treated as unknown (UNK). It returns
    the processed data as numeric IDs, a list of word frequencies, and mappings from tokens to IDs and vice versa.

    Note:
    - It is assumed that `vocab_size` is less than or equal to the total number of unique words in `words`.

    Parameters:
    - words (List[str]): A list of words (tokens) from the dataset.
    - vocab_size (int): The maximum size of the vocabulary, including the 'UNK' token.

    Returns:
    - Tuple containing:
        - List[int]: Numeric identifiers for each word in the input list.
        - List[List[str, int]]: A list where each element is a list with a word and its frequency.
        - Dict[str, int]: A dictionary mapping from tokens to their numeric identifiers.
        - Dict[int, str]: A dictionary mapping from numeric identifiers back to tokens.
    """
    count: List[List[str, int]] = [['UNK', -1]]
    count.extend(collections.Counter(words).most_common(vocab_size - 1))
    token_to_id: Dict[str, int] = {}
    for word, _ in count:
        token_to_id[word] = len(token_to_id)
    data: List[int] = []
    unk_count: int = 0
    for word in words:
        if word in token_to_id:
            index = token_to_id[word]
        else:
            index = 0  # Token ID for 'UNK'
            unk_count += 1
        data.append(index)
    count[0][1] = unk_count
    id_to_token: Dict[int, str] = dict(zip(token_to_id.values(), token_to_id.keys()))

    return data, count, token_to_id, id_to_token


Next, complete the code given below to generate the batches. Do NOT modify any code already written in this cell:

In [ ]:
class Word2VecDataset(Dataset):
    """
    Dataset class for generating batches of training data for the Word2Vec model.

    This class inherits from PyTorch's Dataset class and implements methods for creating batches of training data. It
    generates batches of data and their corresponding labels based on the provided parameters such as batch size, number
    of skips, and skip window.

    Attributes:
    - data (List[int]): The dataset containing word indices.
    - batch_size (int): The number of instances in one batch.
    - num_skips (int): The number of samples to draw in a window.
    - skip_window (int): Determines how many words to consider left and right from a context word.
                         So, skip_window * 2 + 1 = window_size)
    - data_index (int): The index to keep track of the current position in the dataset.

    Methods:
    - __init__: Initializes the Word2Vec dataset with the provided parameters.
    - __len__: Returns the length of the dataset.
    - __getitem__: Retrieves an item (word index) from the dataset at the specified index.
    - reset_index: Resets the current position (`data_index`) to the specified value. Default is 0.
    - generate_batch: Generates a batch of data and labels for training.
    """
    def __init__(self, data: List[int], batch_size=128, num_skips=8, skip_window=4):
        self.data = data
        self.batch_size = batch_size
        self.num_skips = num_skips
        self.skip_window = skip_window
        self.data_index = 0

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        return self.data[index]

    def reset_index(self, idx=0):
        self.data_index = idx

    def generate_batch(self) -> Tuple[torch.Tensor, torch.Tensor]:
        """
        This function generates a batch of training data and corresponding labels for model training. It samples words
        from the dataset to form context-target pairs, where the context words are centered around a target word within
        a skip window. The generated batch consists of input data (target words) and output labels (context words).

        Returns:
        - Tuple[torch.Tensor, torch.Tensor]: A tuple containing the batch of input data (target words) and the batch of
          output labels (context words), both represented as PyTorch tensors.

        Note:
        - This method assumes that the dataset has been preprocessed to represent words as indices.
        - The number of samples drawn in a window (num_skips) determines the number of context words sampled for each
          target word.
        - The skip window specifies the range of context words to consider around each target word.
        """
        batch = np.ndarray(shape=(self.batch_size, 1), dtype=np.int64)
        labels = np.ndarray(shape=(self.batch_size, 1), dtype=np.int64)
        span = 2 * self.skip_window + 1  # [ skip_window target skip_window ]
        buffer = collections.deque(maxlen=span)  # to store a sliding (but fixed-size) window of words

        # TODO #

        #Batch - the size of data to take out
        #ONLY FILL UP ONE BATCH
        #labels - labels for data
        #span - total size of windows plus target
        #buffer where we store our current span

        currentBatchSize = 0
        while currentBatchSize < self.batch_size:
          #Fill batch with data
          if len(buffer) < span:
            buffer.append(self.data[self.data_index])
            self.data_index = self.data_index + 1
          else:
            #Batch has data
            self.data_index = self.data_index - span + 1 #Set index back for next batch
            target = buffer[self.skip_window]
            contextWords = [buffer[i] for i in range(span) if i != self.skip_window] #Get all context words in buffer
            for word in contextWords:
                batch[currentBatchSize, 0] = target #Add target to buffer
                labels[currentBatchSize, 0] = word #Add context word to labels
                currentBatchSize += 1
            if currentBatchSize > self.batch_size:
                  break
            #New batch need to clean buffer
            buffer = collections.deque(maxlen=span)

        #Send to correct device
        batch_tensor = torch.tensor(batch, dtype=torch.int64)
        labels_tensor = torch.tensor(labels, dtype=torch.int64)

        if torch.cuda.is_available():
            batch_tensor = batch_tensor.cuda()
            labels_tensor = labels_tensor.cuda()

        # #Used for testing
        # print(self.data[:128])
        # print(batch)
        # print(labels)
        # print(buffer)
        # return
        return batch_tensor, labels_tensor #Switched to fix device issues should not impact anything

## 3. Model Construction

To revisit the ideas behind the construction of these models and their training process, you may want to look at [these brief notes](https://drive.google.com/file/d/1048NOrttBFvPKDM1PaPSMbaPVrl9b55L/view?usp=drive_link) based on <font color="darkgray">Rong (2016)</font>.

There are two loss functions we will explore in the creation of skip-gram embeddings. In both cases, it is *highly recommended* that you employ the GPU Runtime on Colab.

### Negative Log-Likelihood (NegLL)

In the lectures, we have discussed maximizing the likelihood (hence, the log-likelihood) of the training data. Thus, we can equivalently speak of *minimizing* the negative log-likelihood.

With GPU acceleration on Colab, the training (with default settings and NegLL loss) completes in less than an hour. Attempting the same locally on a laptop may take up to 10-12 hours!

### Negative Sampling, or a Simplified Noise Contrastive Estimation (SiNCE)

This is a slightly different classification task, with a different loss function. This, in fact, is the process we have covered in great detail in the lectures. To ensure a correct implementation, you should consult (as and when needed) the textbook, your own notes, and <font color="darkgray">Mikolov et al. (2013a)</font>. Plus, to understand the background motivation of the *continuous bag of words (CBOW)* model, consult <font color="darkgray">Mikolov et al. (2013b)</font>.


The fundamental idea is to build a classifier that assigns a high probability score to a correct target word, and a low probability score to an incorrect target word. As discussed in our lectures (and by this point, as you have already done with NegLL), we define the classifier using a function (i.e., *hypothesis*) that uses the word vectors of the context and target as free parameters of the model. The important difference in this case is that instead of the entire vocabulary, a sample set of $k$ "noise" words are used for every instance. This process creates an *augmented* instance comprising the true target word and $k$ noise words. With this setup, the training process' goal is to maximize the probability of this augmented instance.

- *Mikolov, T., Sutskever, I., Chen, K., Corrado, G., & Dean, J.* (2013a). [**Distributed Representations of Words and Phrases and their Compositionality**](https://papers.nips.cc/paper/2013/file/9aa42b31882ec039965f3c4923ce901b-Paper.pdf). Advances in Neural Information Processing Systems, 26.
- *Mikolov, T., Chen, K., Corrado, G., & Dean, J.* (2013b). [**Efficient Estimation of Word Representations in Vector Space**](https://arxiv.org/pdf/1301.3781.pdf). arXiv, 1301.3781.
- *Rong, X.* (2016). [**word2vec Parameter Learning Explained**](https://arxiv.org/abs/1411.2738). arXiv, 1411.2738.

*Note on computation*: With GPU acceleration on Colab, training the Word2Vec model with this type of noise contrastive elimination (and default settings) completes in less than 3 hours. *Please do not try this locally on your laptop. It will almost certainly take more than a day!*

In [ ]:
def sigmoid(z: float) -> float:
    return 1 / (1 + torch.exp(-z))

# We define a class 'SkipGramWordVec' derived from 'nn.Module'. This is standard practice when creating models using
# PyTorch. We do this to take advantage of the PyTorch utilities for building neural networks.

class SkipGramWordVec(nn.Module):
    def __init__(self, V, embedding_dim, loss_func, counts, num_neg_samples_per_center = 1):
        """
        Initializes a SkipGramWordVec model.

        Args:
        - V (int): The size of the vocabulary.
        - embedding_dim (int): The dimensionality of word embeddings.
        - loss_func (str): Name of the loss function used for training the model.
        - counts (torch.Tensor or numpy.ndarray): Word frequencies or counts used for negative sampling.
        - num_neg_samples_per_center (int, optional): The number of negative samples per center word. Default is 1.

        Attributes:
        - center_embeddings (torch.nn.Embedding): The embedding layer for center words.
        - context_embeddings (torch.nn.Embedding): The embedding layer for context words.
        - loss_func (str): Name of the loss function used for training.
        - counts (torch.Tensor or numpy.ndarray): Word frequencies or counts used for negative sampling.
        - num_neg_samples_per_center (int): The number of negative samples per center word.

        Initialization:
        - Initializes the embedding layers (`center_embeddings` and `context_embeddings`) with random weights drawn from
          a normal distribution with a mean of 0 and a standard deviation of 1/sqrt(embedding_dim).
        - Clamps the weights of the embedding layers to ensure that embeddings fall within the range [-1, 1].

        Notes:
        - The weights of the embedding layers are initialized randomly and are learnable parameters of the model.
        - The 'counts' parameter is typically provided as a frequency distribution of words in the corpus and is used
          for negative sampling during training.
        """
        super(SkipGramWordVec, self).__init__()

        # The weights of the embedding layers are initialized with random values drawn from a normal distribution with
        # mean 0 and std dev 1/sqrt(embedding_dim). This is a common initialization strategy. Then, a normalization of
        # weights of the embedding layers ensures that the embeddings fall within the range [-1, 1] by clipping values
        # outside this range. This, too, is common (you may have adopted this in the previous assignment as well).
        self.center_embeddings = nn.Embedding(num_embeddings=V, embedding_dim=embedding_dim)
        self.center_embeddings.weight.data.normal_(mean=0, std=1/math.sqrt(embedding_dim))
        self.center_embeddings.weight.data[self.center_embeddings.weight.data<-1] = -1
        self.center_embeddings.weight.data[self.center_embeddings.weight.data>1] = 1

        self.context_embeddings = nn.Embedding(num_embeddings=V, embedding_dim=embedding_dim)
        self.context_embeddings.weight.data.normal_(mean=0, std=1/math.sqrt(embedding_dim))
        self.context_embeddings.weight.data[self.context_embeddings.weight.data < -1] = -1 + 1e-10
        self.context_embeddings.weight.data[self.context_embeddings.weight.data > 1] = 1 - 1e-10

        self.loss_func = loss_func
        self.counts = counts

        self.num_neg_samples_per_center = num_neg_samples_per_center

    def negative_log_likelihood_loss(self, center_word: torch.Tensor, context_word: torch.Tensor) -> torch.Tensor:
        # TODO #
        center_word = center_word.to(self.center_embeddings.weight.device)
        context_word = context_word.to(self.context_embeddings.weight.device)

        #Get embeddings
        centerEmbed = self.center_embeddings(center_word).squeeze(1)
        contextEmbed = self.context_embeddings(context_word).squeeze(1)

        #Get dot prod sum of embeddings
        prod = torch.sum(centerEmbed * contextEmbed, dim=1)

        #Get softmax, using this due to it being numerically stable had issues before with NaN
        logSumProd = torch.logsumexp(torch.matmul(centerEmbed, self.context_embeddings.weight.t()), dim=1) #May need to change .t() to .transpose()

        loss = torch.mean(logSumProd-prod) #avergae between prediccted prod and actual prod
        loss = loss.to(self.center_embeddings.weight.device)

        return loss

    def simple_noise_contrastive_estimation_loss(self, center_word: torch.Tensor,
                                                 context_word: torch.Tensor) -> torch.Tensor:
        """
        Computes the simple noise contrastive estimation loss for a given pair of center and context words.

        Args:
        - center_word (torch.Tensor): Indices of center words in the vocabulary.
        - context_word (torch.Tensor): Indices of context words in the vocabulary.

        Returns:
        - loss (torch.Tensor): Simple noise contrastive estimation loss.

        Notes:
        - This method implements the simple noise contrastive estimation (SiNCE) loss, described also as "negative
          sampling".
        - The input tensors `center_word` and `context_word` should have the same shape.
        - The returned loss tensor represents the average loss over the batch.

        References:
        - "Distributed Representations of Words and Phrases and their Compositionality" by T. Mikolov et al. (2013).
        """
        k = self.num_neg_samples_per_center  # control variable for the number of negative samples per positive sample
        # TODO #
        #epsilon = 1e-7 #For numerical instability

        center_word = center_word.to(self.center_embeddings.weight.device)
        context_word = context_word.to(self.context_embeddings.weight.device)

        #Get embeddings
        centerEmbed = self.center_embeddings(center_word).squeeze(1)
        contextEmbed = self.context_embeddings(context_word).squeeze(1)

        #Get dot prod sum of embeddings
        posProd = torch.sum(centerEmbed * contextEmbed, dim=1)

        #Negitive Sampling
        vocab_size = self.center_embeddings.num_embeddings
        negInd = torch.randint(0, vocab_size, (center_word.size(0), k)).to(self.context_embeddings.weight.device)
        negEmbed = self.context_embeddings(negInd)

        # Calculate the dot product between center word and negative context words
        negative_prod = torch.matmul(negEmbed, centerEmbed.unsqueeze(-1)).squeeze(-1)

        #Using log sigmoid for numerical instablilty, went to office hours with TA and said it should be fine to use in place of given sigmoid
        m = nn.LogSigmoid()
        positive_loss = m(posProd)
        negative_loss = m(-negative_prod).sum(dim=1)
        loss = -(positive_loss + negative_loss).mean()

        return loss

    # The `forward` method is required for any class that inherits from torch.nn.Module
    def forward(self, center_word: torch.Tensor, context_word: torch.Tensor) -> torch.Tensor:
        if self.loss_func == "negative_log_likelihood":
            return self.negative_log_likelihood_loss(center_word, context_word)
        elif self.loss_func == "noise_contrastive_estimation":
            return self.simple_noise_contrastive_estimation_loss(center_word, context_word) #Changed to fix typo in given code was self.negative_sampling(center_word, context_word)
        else:
            raise Exception("Invalid loss function %s" % (self.loss_func))

    def print_closest(self, validation_words: List[int], id_to_token_dict: Dict[int, str], top_k: int = 8) -> None:
        """
        Prints the closest words to the given validation words based on the cosine similarity of word embeddings.

        Args:
        - validation_words (list): List of indices representing validation words.
        - id_to_token_dict (dict): Dictionary mapping word indices to word tokens.
        - top_k (int, optional): Number of closest words to print for each validation word. Default is 8.

        Notes:
        - The embeddings are assumed to be stored in self.center_embeddings.
        """
        print('Printing closest words')
        embeddings = torch.zeros(self.center_embeddings.weight.shape).copy_(self.center_embeddings.weight)
        embeddings = embeddings.data.cpu().numpy()

        validation_ids = validation_words
        norm = np.sqrt(np.sum(np.square(embeddings),axis=1,keepdims=True))
        normalized_embeddings = embeddings/norm
        validation_embeddings = normalized_embeddings[validation_ids]
        similarity = np.matmul(validation_embeddings, normalized_embeddings.T)
        for i in range(len(validation_ids)):
            word = id_to_token_dict[validation_words[i]]
            nearest = (-similarity[i, :]).argsort()[1:top_k+1]
            print(word, [id_to_token_dict[nearest[k]] for k in range(top_k)])

## 4. The Training Loop

In the context of NLP/ML, a **training loop** refers to the iterative process of training a model. Such a loop typically involves the following steps:

1. **Data Loading**: Load the training data, including the input features and corresponding labels.

2. **Model Forward Pass**: Pass the input data through the model to obtain predictions.

3. **Loss Calculation**: Calculate the model's loss based on the difference between its predictions and the actual labels.

4. **Backpropagation**: Propagate the gradients of the loss function back through the model's parameters (i.e., weights and biases) using algorithms like gradient descent.

5. **Parameter Updates**: Update the model parameters using the computed gradients to minimize the loss function.

6. **Validation**: Optionally, evaluate the model on a separate validation/development set to monitor its performance and prevent overfitting. This step can also be done by incorporating a regularization parameter in the loss function (i.e., *regularized loss minimization*).

The above steps are repeated over multiple epochs until the model converges to an optimal solution or reaches its stopping criteria. Such stopping criteria may be predefined (e.g., number of epochs, or the magnitude of change in parameter values).

The training loop is a fundamentally important component of model training. Please use this assignment as a mechanism to thoroughly understand this process.

### Checkpoints

**Checkpoints** are another important concept, esepcially when training large models. It refers to a snapshot of the model's state (including parameters, and any other relevant information) saved at a particular point during training. For large models, it is standard practice to periodically save checkpoints during the training process. This serves multiple purposes:

- It allows us to **resume training** from the last saved checkpoint in case the training process was interrupted, instead of starting from scratch all over again.

- It enables **model evaluation** on the development or test set at different stages of training. By loading a checkpoint and applying it to unseen data, we can evaluate the model as it evolves (over training time/data).

- It helps with **debugging and analysis** because we can examine how certain model parameters are changing, how the loss is varying over epochs, if there are any problems regarding convergence, or if hyperparameters need to be modified.

- It makes **deployment** easier. Once the training is complete, we can use checkpoints to deploy the trained model on new data. This is typically done by loading the final checkpoint at the end of training.

A common abbreviation for "checkpoint" used in ML/NLP is `ckpt`. If you see this abbreviation, is will usually indicate a file or folder containing the saved state of a model at a particular point in training.

The code given below runs the training loop using the models and losses you have implemented earlier in this assignment:

In [ ]:
class Trainer:

    def __init__(self, model, ckpt_save_path, id_to_token_dict: Dict[int, str]=None):
        self.model = model
        self.ckpt_save_path = ckpt_save_path
        self.id_to_token_dict = id_to_token_dict

    def train(self, dataset, max_training_steps, ckpt_interval, validation_words: List[int], validation_interval,
              device="cpu", learning_rate = 0.05):
        optimizer = torch.optim.SGD(params = self.model.parameters(), lr = learning_rate)
        self.model.to(device)

        progress = tqdm(range(max_training_steps))
        training_loss = 0.0
        for t in progress:
            self.model.train()

            optimizer.zero_grad()

            target_words: torch.Tensor
            context_words: torch.Tensor
            target_words, context_words = dataset.generate_batch()  # generate one batch
            #Added make sure on same device
            target_words = target_words.to(device)
            context_words = context_words.to(device)

            loss: torch.Tensor = self.model(target_words, context_words)  # compute the loss
            loss.backward()   # backward pass
            optimizer.step()  # update the parameters
            training_loss += loss.item()  # accumulate the total training loss
            if t:
                progress.set_description("Average loss: %s" % (round(training_loss/(progress.n + 1), 10))) #Chnaged to progress. from t. and training_loss/t.n to (t+1)
            if self.id_to_token_dict and t % validation_interval == 0:
                self.model.print_closest(validation_words, self.id_to_token_dict)
            if t > 0 and t % ckpt_interval == 0:
                self.save_model_checkpoint(t)

    def save_model_checkpoint(self, current_progress):
        torch.save(self.model, "%s/%s.ckpt"%(self.ckpt_save_path, str(current_progress)))


The entire training process is implemented using the `run_training` function. This is the function you should use for various experiments in this assignment.

Importantly, note that this function uses several hyperparameters whose values require experimentation. For example, the batch size, the vocabulary size, the dimension/size of embedding vectors, the number of epochs, the learning rate, etc.

In [ ]:
def create_path(path):
    if not os.path.exists(path):
        os.mkdir(path)
        print ("Created the path: %s" % (path))
id_to_token_dict = {}
token_to_id_dict = {}
def run_training(model_type = 'noise_contrastive_estimation',  # specify the loss function used to train the model
                 learning_rate = 1,  # the learning rate used in training the model
                 num_neg_samples_per_center = 4,  # control for the number of negative samples per center word
                 ckpt_model_path = './ckpt',  # path to the model checkpoint
                 final_model_path = './final_model_ckpt',  # path to the trained model
                 vocab_size = int(1e5),  # size of the vocabulary
                 batch_size = 64,  # batch size: the number of (x,y) pairs used in training
                 num_skips = 2,  # the number of samples to draw in a window
                 skip_window = 1,  # defines how many tokens to consider on both sides of a word
                                   # (i.e., window size = 2 * skip_window + 1)
                 embedding_size = 128,  # size of the embedding vectors
                 ckpt_step_size = 50000,  # number of steps after which model is saved as a checkpoint
                 max_steps = 200001):  # the maximum number of training steps
    ckpt_model_path = f'{ckpt_model_path}_{model_type}/'
    create_path(ckpt_model_path)

    tokens = file_to_tokens("./text8")
    print('Data size', len(tokens))

    global id_to_token_dict, token_to_id_dict
    data, count, token_to_id_dict, id_to_token_dict = build_dataset(tokens, vocab_size)

    print('Top 10 most frequent tokens (+ UNK):', count[:10])
    print('Sample data', data[:20], [id_to_token_dict[i] for i in data[:20]])

    count_dict = dict(count)
    unigram_counts = [count_dict[id_to_token_dict[id]] for id in sorted(list(token_to_id_dict.values()))]

    this_dataset = Word2VecDataset(data, batch_size, num_skips, skip_window)
    targets: torch.Tensor
    contexts: torch.Tensor
    targets, contexts = this_dataset.generate_batch()
    for i in range(10):
        print(targets[i].item(), id_to_token_dict[targets[i].item()], ' -> ',
              contexts[i].item(), id_to_token_dict[contexts[i].item()])
    this_dataset.reset_index()

    valid_size = 16     # Random set of words to evaluate similarity on.
    valid_window = 100  # Only pick dev samples in the head of the distribution.
    valid_examples = np.random.choice(valid_window, valid_size, replace=False)

    embedding_size = embedding_size
    model = SkipGramWordVec(vocab_size, embedding_size, loss_func=model_type, counts=np.array(unigram_counts),
                            num_neg_samples_per_center=num_neg_samples_per_center)
    trainer = Trainer(model, ckpt_model_path, id_to_token_dict)

    this_device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print(f'Device: {this_device}')

    trainer.train(this_dataset, max_steps, ckpt_step_size, validation_words=valid_examples, validation_interval=1000,
                  device=this_device, learning_rate=learning_rate)
    model_path = final_model_path
    create_path(model_path)
    model_filepath = os.path.join(model_path, 'word2vec_%s.model'%(model_type))

    # Using the Pickle serialization format, serialize `token_to_id_dict` and the learned target word embeddings of the
    # model to the file specified by `model_filepath`.
    pickle.dump([token_to_id_dict, model.center_embeddings.weight.detach().cpu().numpy()], open(model_filepath, 'wb'))


Here is a demo code showing how to run the training process. Here, we use significantly fewer training epochs, embedding size, and vocabulary size. This is given only as a way to quickly test the complete implementation. When you run experiments on the skip-gram model, please use values closer to the function defaults shown in the above cell (and *not* the values in this demo code).

Additionally, always observe the average loss being printed as the model gets trained. A correct implementation should, generally, show that the average loss is decreasing.

In [ ]:
run_training(
    model_type = 'noise_contrastive_estimation',  # specify the loss function used to train the model
    learning_rate = 1.0,  # the learning rate used in training the model
    num_neg_samples_per_center = 4,  # control for the number of negative samples per center word
    ckpt_model_path = './demo_ckpt',  # path to the model checkpoint
    final_model_path = './final_demo_ckpt',  # path to the trained model
    vocab_size = int(1e4),  # size of the vocabulary
    batch_size = 128,  # batch size: the number of (x,y) pairs used in training
    embedding_size = 8,  # size of the embedding vectors
    ckpt_step_size = 400,  # number of steps after which model is saved as a checkpoint
    max_steps = 1900  # the maximum number of training steps
)

Data size 17005207
Top 10 most frequent tokens (+ UNK): [['UNK', 1737307], ('the', 1061396), ('of', 593677), ('and', 416629), ('one', 411764), ('in', 372201), ('a', 325873), ('to', 316376), ('zero', 264975), ('nine', 250430)]
Sample data [5234, 3081, 12, 6, 195, 2, 3134, 46, 59, 156, 128, 742, 477, 0, 134, 1, 0, 2, 1, 103] ['anarchism', 'originated', 'as', 'a', 'term', 'of', 'abuse', 'first', 'used', 'against', 'early', 'working', 'class', 'UNK', 'including', 'the', 'UNK', 'of', 'the', 'english']
3081 originated  ->  5234 anarchism
3081 originated  ->  12 as
12 as  ->  3081 originated
12 as  ->  6 a
6 a  ->  12 as
6 a  ->  195 term
195 term  ->  6 a
195 term  ->  2 of
2 of  ->  195 term
2 of  ->  3134 abuse
Device: cuda:0


Average loss: 5.8906602462:   1%|          | 11/1900 [00:00<01:10, 26.70it/s]

Printing closest words
some ['physiological', 'chosen', 'separating', 'responsible', 'popularized', 'dependent', 'wavelengths', 'frankenstein']
from ['labyrinth', 'gui', 'registration', 'back', 'heights', 'aclu', 'prolonged', 'knew']
during ['somewhat', 'tucker', 'painter', 'barriers', 'induction', 'secured', 'penis', 'er']
people ['reason', 'manhattan', 'andrew', 'derby', 'oil', 'paint', 'ordered', 'flown']
for ['whales', 'unix', 'gravitational', 'vessel', 'bands', 'inclined', 'guns', 'data']
nine ['cooking', 'painting', 'drop', 'worn', 'battleships', 'commanded', 'cuts', 'fixed']
but ['decline', 'smell', 'feminists', 'interview', 'shirt', 'rico', 'celebrate', 'participants']
four ['agreements', 'interval', 'goals', 'murray', 'organized', 'spring', 'transmit', 'albert']
d ['dead', 'nevada', 'abbot', 'not', 'america', 'ethanol', 'majesty', 'paint']
often ['romanian', 'medicine', 'feature', 'governors', 'controlling', 'soil', 'ann', 'multiplication']
their ['amounts', 'corporations', 'c

Average loss: 3.4833473018:  54%|█████▍    | 1027/1900 [00:05<00:04, 177.49it/s]

Printing closest words
some ['hall', 'canals', 'governed', 'rivals', 'kubrick', 'morris', 'amos', 'operate']
from ['and', 'for', 'hudson', 'denoted', 'the', 'copied', 'that', 'UNK']
during ['somewhat', 'emerged', 'painter', 'tucker', 'dictatorship', 'induction', 'genesis', 'cameras']
people ['chicken', 'derby', 'oil', 'allied', 'principal', 'who', 'achilles', 'hudson']
for ['from', 'hudson', 'the', 'and', 's', 'in', 'that', 'UNK']
nine ['jpg', 'four', 'two', 'one', 'zero', 'winners', 'iii', 'six']
but ['celebrate', 'really', 'interview', 'serious', 'political', 'gnostic', 'versa', 'edmund']
four ['nine', 'zero', 'herself', 'two', 'iii', 'eight', 'one', 'reinforced']
d ['dead', 'nevada', 'abbot', 'america', 'ethanol', 'not', 'lucid', 'decreasing']
often ['traditional', 'multiplication', 'delay', 'computer', 'surprising', 'companies', 'gehrig', 'curve']
their ['rigid', 'cookies', 'suburb', 'angola', 'mouth', 'on', 'inherent', 'salvation']
are ['december', 'spread', 'enlightenment', 'comm

Average loss: 3.3254908749: 100%|██████████| 1900/1900 [00:09<00:00, 190.05it/s]


Created the path: ./final_demo_ckpt


In order for your assignment to be graded properly, the following are needed:

1. You must train **at least one model for each loss (NegLL and SiNCE)** using the above `run_training` function.

2. The **checkpoints and files must be named appropriately**, reflecting the models details (e.g., the type of loss).

In [ ]:
# TODO : train at least one model with negative log likelihood (NegLL) loss
run_training(
    model_type = 'negative_log_likelihood',  # specify the loss function used to train the model
    learning_rate = 1.0,  # the learning rate used in training the model
    num_neg_samples_per_center = 4,  # control for the number of negative samples per center word
    ckpt_model_path = './NLL_ckpt',  # path to the model checkpoint
    final_model_path = './final_NLL_ckpt',  # path to the trained model
    vocab_size = int(1e5),  # size of the vocabulary
    batch_size = 64,  # batch size: the number of (x,y) pairs used in training
    embedding_size = 128,  # size of the embedding vectors
    ckpt_step_size = 50000,  # number of steps after which model is saved as a checkpoint
    max_steps = 200001  # the maximum number of training steps
)

# model_type = 'noise_contrastive_estimation',  # specify the loss function used to train the model
#                  learning_rate = 1,  # the learning rate used in training the model
#                  num_neg_samples_per_center = 4,  # control for the number of negative samples per center word
#                  ckpt_model_path = './ckpt',  # path to the model checkpoint
#                  final_model_path = './final_model_ckpt',  # path to the trained model
#                  vocab_size = int(1e5),  # size of the vocabulary
#                  batch_size = 64,  # batch size: the number of (x,y) pairs used in training
#                  num_skips = 2,  # the number of samples to draw in a window
#                  skip_window = 1,  # defines how many tokens to consider on both sides of a word
#                                    # (i.e., window size = 2 * skip_window + 1)
#                  embedding_size = 128,  # size of the embedding vectors
#                  ckpt_step_size = 50000,  # number of steps after which model is saved as a checkpoint
#                  max_steps = 200001):  # the maximum number of training steps


Created the path: ./NLL_ckpt_negative_log_likelihood/
Data size 17005207
Top 10 most frequent tokens (+ UNK): [['UNK', 189230], ('the', 1061396), ('of', 593677), ('and', 416629), ('one', 411764), ('in', 372201), ('a', 325873), ('to', 316376), ('zero', 264975), ('nine', 250430)]
Sample data [5234, 3081, 12, 6, 195, 2, 3134, 46, 59, 156, 128, 742, 477, 10572, 134, 1, 27350, 2, 1, 103] ['anarchism', 'originated', 'as', 'a', 'term', 'of', 'abuse', 'first', 'used', 'against', 'early', 'working', 'class', 'radicals', 'including', 'the', 'diggers', 'of', 'the', 'english']
3081 originated  ->  5234 anarchism
3081 originated  ->  12 as
12 as  ->  3081 originated
12 as  ->  6 a
6 a  ->  12 as
6 a  ->  195 term
195 term  ->  6 a
195 term  ->  2 of
2 of  ->  195 term
2 of  ->  3134 abuse
Device: cuda:0


Average loss: 17.2778921127:   0%|          | 1/200001 [00:00<20:18:09,  2.74it/s]

Printing closest words
up ['olimpia', 'afrique', 'gug', 'osterloh', 'collider', 'idiosyncracies', 'preeminence', 'tribunal']
one ['starbucks', 'fejedelem', 'antidepressants', 'serna', 'circumventing', 'attu', 'accusation', 'adonai']
this ['karlstadt', 'gazed', 'geographica', 'frankly', 'inst', 'beeldenstorm', 'artc', 'sahih']
after ['ogata', 'goos', 'sedona', 'europaeus', 'fulvia', 'syntactically', 'otimes', 'vibrio']
often ['nolo', 'appeals', 'weissenfels', 'obligate', 'leda', 'offenbach', 'nutr', 'clemente']
war ['clavell', 'rolando', 'nitrogenous', 'manush', 'arlie', 'afar', 'elisa', 'elven']
more ['superseding', 'investor', 'recolored', 'musicologie', 'doukkala', 'driessen', 'laudare', 'embedded']
state ['separated', 'ornish', 'factitious', 'ietf', 'rewrites', 'anup', 'williams', 'edges']
two ['mercosur', 'pyruvate', 'finalist', 'damadian', 'procuring', 'josey', 'precede', 'tusculum']
if ['capricornus', 'bungay', 'sheets', 'gingivitis', 'hijjah', 'attributes', 'this', 'istvan']
nin

Average loss: 10.7712297199:   0%|          | 989/200001 [00:08<27:28, 120.76it/s]

Printing closest words
up ['be', 'in', 'army', 'at', 'he', 'or', 'union', 'UNK']
one ['five', 'six', 'three', 'seven', 'four', 'two', 'nine', 'eight']
this ['achilles', 'autism', 'an', 'it', 'his', 'UNK', 'or', 'slavery']
after ['on', 'from', 'union', 'had', 'war', 'first', 'at', 'abraham']
often ['an', 'which', 'it', 'museum', 'people', 'toonami', 'abu', 'achilles']
war ['president', 'that', 'all', 'achilles', 'at', 'abraham', 'had', 'state']
more ['on', 'state', 'union', 'from', 'was', 'that', 'in', 'illinois']


...

if ['whenever', 'unless', 'when', 'hence', 'suppose', 'furthermore', 'though', 'moreover']
nine ['eight', 'seven', 'pmid', 'cet', 'oct', 'six', 'gwh', 'four']
been ['be', 'already', 'become', 'recently', 'newly', 'previously', 'successfully', 'publicly']
many ['several', 'numerous', 'some', 'various', 'these', 'all', 'certain', 'few']
i ['we', 'iii', 'you', 'iv', 'mail', 'yes', 'ii', 'mathbf']
would ['could', 'might', 'will', 'should', 'must', 'may', 'cannot', 'shall']
but ['however', 'nor', 'moreover', 'although', 'similarly', 'furthermore', 'additionally', 'likewise']


Average loss: 6.9170800453: 100%|█████████▉| 199990/200001 [27:59<00:00, 119.29it/s]

Printing closest words
up ['off', 'down', 'together', 'forth', 'away', 'out', 'back', 'forward']
one ['ers', 'cet', 'cam', 'cvn', 'unassigned', 'two', 'radios', 'six']
this ['which', 'neither', 'itself', 'it', 'another', 'the', 'some', 'sensitive']
after ['before', 'despite', 'during', 'afterwards', 'thereafter', 'when', 'without', 'following']
often ['frequently', 'usually', 'sometimes', 'commonly', 'typically', 'generally', 'normally', 'rarely']
war ['wars', 'cia', 'champion', 'crisis', 'factbook', 'battle', 'liberties', 'campaign']
more ['less', 'most', 'highly', 'faster', 'fairly', 'increasingly', 'quite', 'very']
state ['city', 'government', 'abet', 'district', 'provincial', 'charter', 'department', 'labour']
two ['four', 'three', 'five', 'cet', 'six', 'unassigned', 'cyg', 'kg']
if ['whenever', 'unless', 'when', 'hence', 'suppose', 'furthermore', 'moreover', 'though']
nine ['eight', 'seven', 'pmid', 'cet', 'oct', 'six', 'gwh', 'ff']
been ['be', 'already', 'become', 'recently', 'ne

Average loss: 6.9170800453: 100%|██████████| 200001/200001 [28:00<00:00, 119.03it/s]


Created the path: ./final_NLL_ckpt


In [ ]:
# TODO : train at least one model with negative sampling, i.e., the simplified noise contrastive mining (SiNCE) loss
run_training(
    model_type = 'noise_contrastive_estimation',  # specify the loss function used to train the model
    learning_rate = 1.0,  # the learning rate used in training the model
    num_neg_samples_per_center = 4,  # control for the number of negative samples per center word
    ckpt_model_path = './SiNCE_ckpt',  # path to the model checkpoint
    final_model_path = './final_SiNCE_ckpt',  # path to the trained model
    vocab_size = int(1e5),  # size of the vocabulary
    batch_size = 64,  # batch size: the number of (x,y) pairs used in training
    embedding_size = 128,  # size of the embedding vectors
    ckpt_step_size = 50000,  # number of steps after which model is saved as a checkpoint
    max_steps = 200001  # the maximum number of training steps
)

# model_type = 'noise_contrastive_estimation',  # specify the loss function used to train the model
#                  learning_rate = 1,  # the learning rate used in training the model
#                  num_neg_samples_per_center = 4,  # control for the number of negative samples per center word
#                  ckpt_model_path = './ckpt',  # path to the model checkpoint
#                  final_model_path = './final_model_ckpt',  # path to the trained model
#                  vocab_size = int(1e5),  # size of the vocabulary
#                  batch_size = 64,  # batch size: the number of (x,y) pairs used in training
#                  num_skips = 2,  # the number of samples to draw in a window
#                  skip_window = 1,  # defines how many tokens to consider on both sides of a word
#                                    # (i.e., window size = 2 * skip_window + 1)
#                  embedding_size = 128,  # size of the embedding vectors
#                  ckpt_step_size = 50000,  # number of steps after which model is saved as a checkpoint
#                  max_steps = 200001):  # the maximum number of training steps


Created the path: ./SiNCE_ckpt_noise_contrastive_estimation/
Data size 17005207
Top 10 most frequent tokens (+ UNK): [['UNK', 189230], ('the', 1061396), ('of', 593677), ('and', 416629), ('one', 411764), ('in', 372201), ('a', 325873), ('to', 316376), ('zero', 264975), ('nine', 250430)]
Sample data [5234, 3081, 12, 6, 195, 2, 3134, 46, 59, 156, 128, 742, 477, 10572, 134, 1, 27350, 2, 1, 103] ['anarchism', 'originated', 'as', 'a', 'term', 'of', 'abuse', 'first', 'used', 'against', 'early', 'working', 'class', 'radicals', 'including', 'the', 'diggers', 'of', 'the', 'english']
3081 originated  ->  5234 anarchism
3081 originated  ->  12 as
12 as  ->  3081 originated
12 as  ->  6 a
6 a  ->  12 as
6 a  ->  195 term
195 term  ->  6 a
195 term  ->  2 of
2 of  ->  195 term
2 of  ->  3134 abuse
Device: cuda:0


  0%|          | 0/200001 [00:00<?, ?it/s]

Printing closest words
state ['meccans', 'scratched', 'lisgar', 'novelizations', 'klt', 'knitter', 'kintyre', 'symons']
were ['goumiere', 'believable', 'nchen', 'peremptory', 'lapithae', 'yu', 'kirchhoff', 'initiating']
a ['barbat', 'naha', 'classicus', 'widukind', 'sah', 'cognomen', 'fencers', 'ornithischians']
are ['motorola', 'croagh', 'cysteine', 'halfback', 'charney', 'perturbations', 'davros', 'garages']
however ['educaci', 'diapers', 'engaging', 'detritus', 'preys', 'monge', 'cyberchase', 'kissed']
see ['metonic', 'intermarry', 'leq', 'tudors', 'rectangle', 'demobilization', 'yaque', 'encumbered']
while ['asf', 'brevet', 'turkiye', 'mensagem', 'aphanitic', 'erzherzog', 'waitakere', 'tcat']


Average loss: 15.6100629568:   0%|          | 1/200001 [00:00<17:23:34,  3.19it/s]

would ['stroustrup', 'siri', 'greenery', 'distinctives', 'clough', 'perros', 'ghent', 'hamada']
with ['chuter', 'devos', 'triplanetary', 'lankavatara', 'cweb', 'salute', 'ninfa', 'fourmilab']
new ['vanbrugh', 'midshipman', 'kyu', 'nuys', 'kirghiz', 'verified', 'navarrese', 'pleased']
two ['abdominis', 'graham', 'refraining', 'uncaring', 'lechmere', 'semicircle', 'party', 'giant']
called ['neptunium', 'originals', 'aedificiis', 'valerian', 'vaphio', 'drakes', 'scarface', 'holinshed']
will ['haydn', 'usurpation', 'noise', 'trentham', 'hamza', 'primrose', 'ove', 'beachy']
for ['rzenich', 'oratorios', 'evanescence', 'pixels', 'minoris', 'litterbox', 'swimsuit', 'issn']
which ['utilized', 'bartholin', 'minuit', 'stake', 'cpk', 'sopranino', 'guia', 'verne']
but ['screens', 'skoll', 'greystone', 'vied', 'sucrose', 'katrine', 'intercepted', 'onega']


...

with ['between', 'under', 'when', 'before', 'through', 'became', 'using', 'while']
new ['old', 'city', 'began', 'western', 'modern', 'member', 'parallel', 'actress']
two ['three', 'four', 'five', 'one', 'six', 'seven', 'isbn', 'years']
called ['received', 'served', 'capture', 'combined', 'move', 'fourth', 'concept', 'pronounced']
will ['would', 'could', 'may', 'can', 'must', 'might', 'should', 'cannot']
for ['without', 'through', 'because', 'against', 'using', 'no', 'while', 'scientific']
which ['it', 'there', 'however', 'what', 'that', 'this', 'typically', 'they']
but ['however', 'within', 'because', 'generally', 'although', 'even', 'still', 'using']


Average loss: 1.2041646633: 100%|█████████▉| 199989/200001 [23:19<00:00, 157.97it/s]

Printing closest words
state ['member', 'province', 'third', 'country', 'career', 'town', 'release', 'former']
were ['being', 'are', 'have', 'had', 'when', 'later', 'under', 'often']
a ['any', 'another', 'very', 'each', 'single', 'every', 'range', 'color']
are ['were', 'have', 'being', 'used', 'such', 'although', 'become', 'but']
however ['within', 'although', 'because', 'using', 'generally', 'though', 'but', 'across']
see ['though', 'special', 'traditional', 'physical', 'current', 'ancient', 'modern', 'jewish']
while ['although', 'within', 'during', 'under', 'however', 'because', 'though', 'among']
would ['will', 'may', 'could', 'can', 'must', 'should', 'might', 'cannot']
with ['between', 'under', 'when', 'before', 'became', 'through', 'using', 'around']
new ['old', 'city', 'began', 'western', 'modern', 'member', 'parallel', 'traditional']
two ['three', 'four', 'five', 'six', 'seven', 'one', 'isbn', 'eight']
called ['received', 'served', 'capture', 'concept', 'fact', 'given', 'move', 

Average loss: 1.2041646633: 100%|██████████| 200001/200001 [23:20<00:00, 142.83it/s]


Created the path: ./final_SiNCE_ckpt


## 5. Model Evaluation: The Analogy Test

To test the quality of the embeddings, we will resort to the **word analogy test**. This comprises "fill in the blank" questions of the form

- *A* is to *B* as *C* is to __

We know from previous research that static embeddings like Word2Vec are not great at capturing all kinds of similarity. So, we are not aiming for 100% accuracy! But, we do want to compare the embeddings generated in this assignment against the best embeddings in this category.

As such, our testing will consist of comparisons of among our models as well as against several GloVe embeddings.

Therefore, let us first get the pretrained GloVe embeddings:

In [ ]:
!wget https://nlp.stanford.edu/data/glove.6B.zip
!unzip glove*.zip

--2024-04-12 23:17:59--  https://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2024-04-12 23:17:59--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6B.zip        100%[===================>] 822.24M  5.01MB/s    in 2m 39s  

2024-04-12 23:20:39 (5.16 MB/s) - ‘glove.6B.zip’ saved [862182613/862182613]

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflatin

Unzipping above gives you the following embeddings (the number in each file's name indicated the dimension of the embedding vectors):

- glove.6B.50d.txt
- glove.6B.100d.txt
- glove.6B.200d.txt
- glove.6B.300d.txt

Next, we will use the `gensim` library to convert these pretrained GloVe embeddings into the standard format, so that we can readily use them.

In [ ]:
from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.models import KeyedVectors

glove_input_file = 'glove.6B.50d.txt'
word2vec_output_file = 'glove2vec.txt'
glove2word2vec(glove_input_file, word2vec_output_file)

# Example of ready use of these pretrained GloVe embeddings:
model = KeyedVectors.load_word2vec_format('glove2vec.txt')
print(model.most_similar('python'))
# # >>> [('reticulated', 0.6916365623474121), ('spamalot', 0.6635736227035522), ('php', 0.6414496898651123), ...]

<ipython-input-22-80714ee69ea0>:6: DeprecationWarning: Call to deprecated `glove2word2vec` (KeyedVectors.load_word2vec_format(.., binary=False, no_header=True) loads GLoVE text vectors.).
  glove2word2vec(glove_input_file, word2vec_output_file)


[('reticulated', 0.6916365623474121), ('spamalot', 0.6635736227035522), ('php', 0.6414496302604675), ('owl', 0.6301496028900146), ('mouse', 0.6275478601455688), ('reticulatus', 0.6274471282958984), ('perl', 0.6267575025558472), ('monkey', 0.6207211017608643), ('monty', 0.60793536901474), ('scripting', 0.6041731834411621)]


The `most_similar` method mentioned above is going to be really important in our testing framework. A simple function to solve the word analogy problem using `gensim` and the GloVe embeddings is given to you below:

In [ ]:
def solve_analogy_glove(a: str, b: str, c: str, model: KeyedVectors) -> Union[Tuple[str, float], Tuple[str, None]]:
    """
    Solves the analogy: a is to b as c is to ____ using the provided model.

    Parameters:
    - a (str): The word a in the analogy.
    - b (str): The word b in the analogy, related to word a.
    - c (str): The word c in the analogy, seeking its counterpart.
    - model (gensim.models.KeyedVectors): The loaded word vector model.

    Returns:
    - str: The word that completes the analogy.
    - float: The similarity score of the predicted word.
    """
    try:
        # positive=['b', 'c'], negative=['a'] is equivalent to: b - a + c
        # most_similar returns a list of tuples (word, similarity)
        result = model.most_similar(positive=[b, c], negative=[a], topn=1)
        return result[0][0], result[0][1]  # Return the top result word and its similarity score
    except KeyError as e:
        # This exception block handles the case where any of 'a', 'b', or 'c' is not in the vocabulary
        return str(e), None


Your task is to write a similar function that takes in the **final saved model checkpoint** (instead of the `KeyedVector` object, as done above using `gensim`).

In [ ]:
ckpt_path = '/content/NLL_ckpt_negative_log_likelihood/200000.ckpt'  # TODO: replace this with the actual path to the final checkpoint of YOUR
                                       # Word2Vec model
loaded_word2vec_model = torch.load(ckpt_path)

# ckpt_path = '/content/final_NLL_ckpt/word2vec_negative_log_likelihood.model'  # TODO: replace this with the actual path to the final checkpoint of YOUR
#                                        # Word2Vec model
# # loaded_word2vec_model = torch.load(ckpt_path)
# loaded_word2vec_model = pickle.load(open(ckpt_path, 'rb'))

def solve_analogy_word2vec(a: str, b: str, c: str, model: nn.Module) -> Union[Tuple[str, float], Tuple[str, None]]:
    """
    Solves the analogy: a is to b as c is to ____ using the provided model.

    Parameters:
    - a (str): The word a in the analogy.
    - b (str): The word b in the analogy, related to word a.
    - c (str): The word c in the analogy, seeking its counterpart.
    - model (torch.nn.Module): The PyTorch model providing word embeddings.

    Returns:
    - Tuple[str, float] or Tuple[str, None]: A tuple containing the word that completes the analogy
      and its similarity score, or a tuple with an error message and None if any of 'a', 'b', or 'c'
      is not in the vocabulary.
    """
    # TODO

    try:
      #Get token ids
      idA = token_to_id_dict.get(a)
      idB = token_to_id_dict.get(b)
      idC = token_to_id_dict.get(c)

      if idA is None or idB is None or idC is None:
          raise KeyError("Not in vocabulary")

      #Get embeddings and id tensors
      idA_tensor = torch.tensor([idA], dtype=torch.long, device=model.center_embeddings.weight.device)
      idB_tensor = torch.tensor([idB], dtype=torch.long, device=model.center_embeddings.weight.device)
      idC_tensor = torch.tensor([idC], dtype=torch.long, device=model.center_embeddings.weight.device)

      embedA = model.center_embeddings(idA_tensor)
      embedB = model.center_embeddings(idB_tensor)
      embedC = model.center_embeddings(idC_tensor)

      #Need to find what c + (b-a) is to get d
      analogy = embedB - embedA + embedC

      #Compute cosine similarity to find nearest
      cosineSimilarity =  torch.nn.functional.cosine_similarity
      cosineSimilarity = cosineSimilarity(analogy, model.center_embeddings.weight, dim=-1)

      mostSimInd = torch.argmax(cosineSimilarity).item()
      #Get most similar word
      mostSimWord = next(w for w, id in token_to_id_dict.items() if id == mostSimInd)

      score = cosineSimilarity[mostSimInd].item()

      return mostSimWord, score

    except KeyError as e:
        return str(e), None

Now that we have two functions (one using GloVe, and one using your model), we are ready for the evaluation.

For this step, note that your model is a PyTorch `nn.Module` object, and does not have a built-in `most_similar` function like Gensim's `KeyedVectors`. You probably noticed this already, while implementing the `solve_analogy_word2vec` function. But we highlight this again, because now you have to measure similarities *very* often, and for a large file. So, it has to be very efficiently implemented!

> Running `solve_analogy_glove` on every instance in the `word2vec-word-analogies.txt` file take around 9m 35s on Colab using the CPU runtime. While this is not easy to achieve, it still provides an approximation of how efficient your code should be (for example, if it takes 30m, that is probably ok; but if it takes 3 hours, that indicates obvious room for improvement!)

With this in mind:

1. Complete the function `compute_accuracy`

2. Load the file `word2vec-word-analogies.txt` given to you as part of the assignment, and analyze its structure. You will realize that a few lines seem to have a distinct format, and its time to use some "regular expression" skills to figure out how many lines are in that special format. Once you can collect all such lines, their meaning will also become clear.

3. Complete the functions `evaluate_glove` and `evaluate_word2vec`. This is very similar to `compute_accuracy`, except that you must report the accuracy for each category separately.



In [ ]:
import re #Imported re as we are instructed to use our "regular expression" skills

def compute_accuracy(file_path: str, model: Union[KeyedVectors, nn.Module]) -> float:
    """
    Computes the accuracy of the provided model on the word analogy task.

    Parameters:
    - file_path (str): The file path to the analogies file.
    - model (Union[KeyedVectors, ModuleType]): The word embedding model. It can be either a Gensim KeyedVectors object
      or a PyTorch nn.Module.

    Returns:
    - float: The accuracy of the model on the analogies task.
      Accuracy is computed as the ratio of correctly predicted analogies to the total number of analogies evaluated.
    """
    # TODO

    #Finding total accuracy
    #Read line and parse into 4 variables, per line a b c d
    #Call solve and check if it matches d
    #Keep track of success versus total

    #Pull all valid lines and put in tuples of what should be 4 words
    analogies = []
    with open(file_path, 'r') as file:
      for line in file:
        if not re.match(r'^:', line):
          analogy = tuple(line.strip().split())
          analogies.append(analogy)

    totalAnalogies = len(analogies)
    print(totalAnalogies)


    correctAnalogies = 0
    missingAnalogies = 0
    incorrectAnalogies= 0
    #For all analogies in file
    for analogy in analogies:
      a, b, c, d = analogy
      a = a.lower()
      b = b.lower()
      c = c.lower()
      d = d.lower()
      #Is gloVe
      if isinstance(model,KeyedVectors):
        predictedWord, _ = solve_analogy_glove(a, b, c, model) #Solve
        if predictedWord is not None and predictedWord == d:
          correctAnalogies +=1
        elif len(predictedWord.split(' ')) > 1:
          missingAnalogies +=1
        else:
          incorrectAnalogies += 1
      else: #Should be our model
        predictedWord, _ = solve_analogy_word2vec(a, b, c, model) #solve
        #print(f'{d} or {predictedWord}')
        if predictedWord is not None and predictedWord == d:
          correctAnalogies +=1
          #print(f'Correct: {d}, Guessed: {predictedWord}')
        elif len(predictedWord.split(' ')) > 1:
          missingAnalogies +=1
        else:
          incorrectAnalogies +=1
    print(f"Correct {correctAnalogies}, Missing {missingAnalogies}, Incorrect {incorrectAnalogies}")
    return correctAnalogies/totalAnalogies

# Example use:
# file_path = 'word2vec-word-analogies.txt'
# model = KeyedVectors.load_word2vec_format('glove2vec.txt')
# accuracy = compute_accuracy(file_path, model)
# print("Accuracy: %.3f" % accuracy)
# # >>> Accuracy: 0.462

def evaluate_glove(file_path: str, model: KeyedVectors) -> None:
    """
    Computes the accuracy of the provided model on the word analogy task, separately for every relation category, and
    prints the result in a neat tabular structure as follows in decreasing order of accuracy:

    Category                     Accuracy
    capital-common-countries     0.488
    city-in-state                0.472
    gram1-adjective-to-adverb    0.471
    ...                          ...
    ...                          ...
    """
    # TODO

    #Copy accuracy but keep track of category

    analogies = []
    categoryDict = {} #What categories we have seen
    currentCategory = ''
    with open(file_path, 'r') as file:
      for line in file:
        if not re.match(r'^:', line):
          analogy = tuple( line.strip().split())
          analogies.append((currentCategory,analogy)) #Store with analogy
        else:
          splitCategory = line.strip().split()
          currentCategory = splitCategory[-1]
          if currentCategory not in categoryDict:
            categoryDict[currentCategory] = [0,0] #New category, first is success second is total

    totalAnalogies = len(analogies)
    correctAnalogies = 0
    for category, analogy in analogies:
      a, b, c, d = analogy
      a = a.lower()
      b = b.lower()
      c = c.lower()
      d = d.lower()
      predictedWord, _ = solve_analogy_glove(a, b, c, model)
      if predictedWord is not None and predictedWord == d:
        categoryDict[category][0] += 1
      categoryDict[category][1] += 1


    #Compute accuracy for each category with stored values
    print('Category                     Accuracy')
    sortedCategories = sorted(categoryDict.keys(), key=lambda x: categoryDict[x][0] / categoryDict[x][1], reverse=True)
    for category in sortedCategories:
            accuracy = categoryDict[category][0] / categoryDict[category][1]
            print(f"{category:<30} {accuracy:0.3f}") #Formating and print


def evaluate_word2vec(file_path: str, model: nn.Module) -> None:
    """
    Computes the accuracy of the provided model on the word analogy task, separately for every relation category, and
    prints the result in a neat tabular structure as follows in decreasing order of accuracy:

    Category                     Accuracy
    city-in-state                0.344
    gram1-adjective-to-adverb    0.339
    ...                          ...
    ...                          ...
    """
    # TODO

    #Copy accuracy but keep track of category

    analogies = []
    categoryDict = {} #What categories we have seen
    currentCategory = ''
    with open(file_path, 'r') as file:
      for line in file:
        if not re.match(r'^:', line):
          analogy = tuple( line.strip().split())
          analogies.append((currentCategory,analogy)) #Store with analogy
        else:
          splitCategory = line.strip().split()
          currentCategory = splitCategory[-1]
          if currentCategory not in categoryDict:
            categoryDict[currentCategory] = [0,0] #New category, first is success second is total

    totalAnalogies = len(analogies)
    correctAnalogies = 0
    for category, analogy in analogies:
      a, b, c, d = analogy
      a = a.lower()
      b = b.lower()
      c = c.lower()
      d = d.lower()
      predictedWord, _ = solve_analogy_word2vec(a, b, c, model)
      if predictedWord is not None and predictedWord == d:
        categoryDict[category][0] += 1
      categoryDict[category][1] += 1


    #Compute accuracy for each category with stored values
    print('Category                     Accuracy')
    sortedCategories = sorted(categoryDict.keys(), key=lambda x: categoryDict[x][0] / categoryDict[x][1], reverse=True)
    for category in sortedCategories:
            accuracy = categoryDict[category][0] / categoryDict[category][1]
            print(f"{category:<30} {accuracy:0.3f}") #Formating and print

### 5.1 Hyperparameters for NegLL and SiNCE models

Make sure you tune the following hyperparameters:

- Number of Neg samples (varying from 1 to 5)
- Learning Rate (varying from 0.01 to 5)
- Window size (varying from 1 to 10)

Run `evaluate_word2vec` on at least two hyperparameter combinations for NegLL and at two hyperparameter combinations for SiNCE.

Also run `evaluate_glove` (i.e., with the pretrained GloVe embeddings). The only hyperparameter here is the embedding size, i.e., the dimension of the vectors (50, 100, 200, 300).

**Do NOT remove the results of these runs. A lot of your grading depends on the results displayed here!**

* In order to ensure that the results are ALL shown, you may want to add one code cell per run (i.e., one cell for each model-hyperparameter combination).


In [ ]:
#Glove with 50 dimensional vectors
file_path = 'word2vec-word-analogies.txt'
model = KeyedVectors.load_word2vec_format('/content/glove2vec.txt')
evaluate_glove(file_path, model)

Category                     Accuracy
gram6-nationality-adjective    0.860
capital-common-countries       0.792
family                         0.690
capital-world                  0.685
gram8-plural                   0.599
gram3-comparative              0.518
gram5-present-participle       0.416
gram7-past-tense               0.375
gram9-plural-verbs             0.344
gram4-superlative              0.286
city-in-state                  0.153
gram1-adjective-to-adverb      0.152
gram2-opposite                 0.095
currency                       0.083


In [ ]:
#Glove with 100 dimensional vectors
file_path = 'word2vec-word-analogies.txt'
glove_input_file = 'glove.6B.100d.txt'
word2vec_output_file = 'glove2vec100d.txt'
glove2word2vec(glove_input_file, word2vec_output_file)
model = KeyedVectors.load_word2vec_format('/content/glove2vec100d.txt')
evaluate_glove(file_path, model)

<ipython-input-20-f5b3d9261d54>:4: DeprecationWarning: Call to deprecated `glove2word2vec` (KeyedVectors.load_word2vec_format(.., binary=False, no_header=True) loads GLoVE text vectors.).
  glove2word2vec(glove_input_file, word2vec_output_file)


Category                     Accuracy
capital-common-countries       0.939
capital-world                  0.889
gram6-nationality-adjective    0.879
family                         0.816
gram3-comparative              0.791
gram8-plural                   0.720
gram5-present-participle       0.695
gram9-plural-verbs             0.584
gram7-past-tense               0.554
gram4-superlative              0.543
city-in-state                  0.308
gram1-adjective-to-adverb      0.244
gram2-opposite                 0.201
currency                       0.142


In [ ]:
#Glove with 200 dimensional vectors
file_path = 'word2vec-word-analogies.txt'
glove_input_file = 'glove.6B.200d.txt'
word2vec_output_file = 'glove2vec200d.txt'
glove2word2vec(glove_input_file, word2vec_output_file)
model = KeyedVectors.load_word2vec_format('/content/glove2vec200d.txt')
evaluate_glove(file_path, model)

<ipython-input-21-d36048b6488f>:4: DeprecationWarning: Call to deprecated `glove2word2vec` (KeyedVectors.load_word2vec_format(.., binary=False, no_header=True) loads GLoVE text vectors.).
  glove2word2vec(glove_input_file, word2vec_output_file)


Category                     Accuracy
capital-common-countries       0.947
capital-world                  0.946
gram6-nationality-adjective    0.925
gram3-comparative              0.865
family                         0.856
gram8-plural                   0.774
gram4-superlative              0.698
gram5-present-participle       0.683
gram7-past-tense               0.602
gram9-plural-verbs             0.598
city-in-state                  0.500
gram1-adjective-to-adverb      0.254
gram2-opposite                 0.227
currency                       0.172


In [ ]:
#Glove with 300 dimensional vectors
file_path = 'word2vec-word-analogies.txt'
glove_input_file = 'glove.6B.300d.txt'
word2vec_output_file = 'glove2vec300d.txt'
glove2word2vec(glove_input_file, word2vec_output_file)
model = KeyedVectors.load_word2vec_format('/content/glove2vec300d.txt')
evaluate_glove(file_path, model)

<ipython-input-22-f49a0131da6b>:4: DeprecationWarning: Call to deprecated `glove2word2vec` (KeyedVectors.load_word2vec_format(.., binary=False, no_header=True) loads GLoVE text vectors.).
  glove2word2vec(glove_input_file, word2vec_output_file)


Category                     Accuracy
capital-world                  0.960
capital-common-countries       0.949
gram6-nationality-adjective    0.926
family                         0.881
gram3-comparative              0.881
gram8-plural                   0.781
gram4-superlative              0.722
gram5-present-participle       0.700
gram7-past-tense               0.612
city-in-state                  0.593
gram9-plural-verbs             0.585
gram2-opposite                 0.273
gram1-adjective-to-adverb      0.226
currency                       0.158


In [ ]:
#Default Params NLL
file_path = 'word2vec-word-analogies.txt'
model = torch.load('/content/NLL_ckpt_negative_log_likelihood/200000.ckpt')
evaluate_word2vec(file_path, model)

Category                     Accuracy
gram3-comparative              0.011
gram9-plural-verbs             0.007
family                         0.004
gram7-past-tense               0.001
gram5-present-participle       0.001
capital-common-countries       0.000
capital-world                  0.000
currency                       0.000
city-in-state                  0.000
gram1-adjective-to-adverb      0.000
gram2-opposite                 0.000
gram4-superlative              0.000
gram6-nationality-adjective    0.000
gram8-plural                   0.000


In [ ]:
#Default Params SiNCE
file_path = 'word2vec-word-analogies.txt'
model = torch.load('/content/SiNCE_ckpt_noise_contrastive_estimation/200000.ckpt')
evaluate_word2vec(file_path, model)

Category                     Accuracy
capital-common-countries       0.000
capital-world                  0.000
currency                       0.000
city-in-state                  0.000
family                         0.000
gram1-adjective-to-adverb      0.000
gram2-opposite                 0.000
gram3-comparative              0.000
gram4-superlative              0.000
gram5-present-participle       0.000
gram6-nationality-adjective    0.000
gram7-past-tense               0.000
gram8-plural                   0.000
gram9-plural-verbs             0.000


In [ ]:
#Changed learning rate to 0.5, neg samples to 3, and window size to 2
run_training(
    model_type = 'negative_log_likelihood',  # specify the loss function used to train the model
    learning_rate = 0.5,  # the learning rate used in training the model
    num_neg_samples_per_center = 3,  # control for the number of negative samples per center word
    ckpt_model_path = './NLL_Updated_Params_1_ckpt',  # path to the model checkpoint
    final_model_path = './final_NLL_Updated_Params_1_ckpt',  # path to the trained model
    vocab_size = int(1e5),  # size of the vocabulary
    batch_size = 64,  # batch size: the number of (x,y) pairs used in training
    embedding_size = 128,  # size of the embedding vectors
    ckpt_step_size = 50000,  # number of steps after which model is saved as a checkpoint
    max_steps = 200001,  # the maximum number of training steps
    skip_window = 2
)



Created the path: ./NLL_Updated_Params_1_ckpt_negative_log_likelihood/
Data size 17005207
Top 10 most frequent tokens (+ UNK): [['UNK', 189230], ('the', 1061396), ('of', 593677), ('and', 416629), ('one', 411764), ('in', 372201), ('a', 325873), ('to', 316376), ('zero', 264975), ('nine', 250430)]
Sample data [5234, 3081, 12, 6, 195, 2, 3134, 46, 59, 156, 128, 742, 477, 10572, 134, 1, 27350, 2, 1, 103] ['anarchism', 'originated', 'as', 'a', 'term', 'of', 'abuse', 'first', 'used', 'against', 'early', 'working', 'class', 'radicals', 'including', 'the', 'diggers', 'of', 'the', 'english']
12 as  ->  5234 anarchism
12 as  ->  3081 originated
12 as  ->  6 a
12 as  ->  195 term
6 a  ->  3081 originated
6 a  ->  12 as
6 a  ->  195 term
6 a  ->  2 of
195 term  ->  12 as
195 term  ->  6 a
Device: cuda:0


  0%|          | 0/200001 [00:00<?, ?it/s]

Printing closest words
new ['cold', 'treatise', 'kansi', 'webserver', 'menorah', 'beauties', 'streamlining', 'adn']
their ['hohe', 'ondimba', 'rog', 'ondes', 'nondeterministic', 'fringing', 'saturninus', 'assaulting']
UNK ['worshipper', 'kapela', 'josep', 'dbe', 'cardboard', 'coot', 'lengthen', 'luhn']
and ['indelicate', 'myra', 'ranic', 'ultrastructure', 'subsists', 'astyanax', 'spikes', 'sidestep']
an ['groundless', 'microscopes', 'anthromorphic', 'surratt', 'inlaid', 'artorius', 'engravings', 'disallowing']


Average loss: 28.7923722267:   0%|          | 1/200001 [00:00<18:16:12,  3.04it/s]

when ['basement', 'grieves', 'guianese', 'philosophers', 'brauron', 'cnet', 'ridgeline', 'maupin']
also ['peiper', 'memorialized', 'protogalaxies', 'untested', 'tetrahedrally', 'jannaeus', 'bullis', 'melina']
are ['brigitte', 'taizong', 'xilinx', 'tagg', 'unionization', 'sanctification', 'karr', 'cruise']
be ['ernmas', 'metre', 'rachael', 'paddle', 'rowman', 'first', 'kojeve', 'modugno']
not ['mainyu', 'injects', 'rightfully', 'angelic', 'adriaan', 'discotic', 'tanque', 'folksongs']
use ['divinely', 'nuv', 'mansart', 'outlandish', 'deploying', 'amedeo', 'transpositions', 'eurythmics']
there ['mailing', 'alcopop', 'hypertextual', 'anycast', 'unequally', 'dff', 'exalting', 'observatory']
history ['sarcoma', 'densely', 'mikha', 'bundesl', 'picking', 'nest', 'experimented', 'schnabel']
system ['cartier', 'speculators', 'reunion', 'dacoit', 'bisected', 'giovani', 'laurentianus', 'causation']
if ['marcescens', 'acme', 'subband', 'dormitory', 'rizzo', 'masatake', 'salom', 'shrapnel']
th ['gor

...

also ['still', 'mainly', 'which', 'often', 'now', 'actually', 'always', 'sometimes']
are ['were', 'is', 'being', 'include', 'various', 'themselves', 'have', 'exist']
be ['being', 'been', 'often', 'usually', 'have', 'generally', 'never', 'himself']
not ['never', 'still', 'usually', 'if', 'actually', 'nor', 'always', 'without']
use ['form', 'cause', 'using', 'value', 'brain', 'existence', 'share', 'energy']
there ['longer', 'indeed', 'therefore', 'true', 'still', 'actually', 'it', 'neither']
history ['bulgaria', 'administrative', 'stock', 'religion', 'demographics', 'afghanistan', 'atlanta', 'literature']
system ['systems', 'software', 'style', 'computer', 'brain', 'design', 'type', 'building']
if ['then', 'because', 'thus', 'therefore', 'simply', 'without', 'unless', 'when']
th ['bc', 'nd', 'st', 'ad', 'twentieth', 'rd', 'bce', 'mid']


Average loss: 7.3603510376: 100%|█████████▉| 199991/200001 [29:31<00:00, 85.19it/s]

Printing closest words
new ['county', 'york', 'capital', 'city', 'dance', 'arbor', 'machine', 'future']
their ['its', 'his', 'our', 'your', 'her', 'the', 'both', 'these']
UNK ['green', 'prince', 'peter', 'blue', 'joseph', 'family', 'daughter', 'png']
and ['while', 'but', 'or', 'against', 'bring', 'through', 'causing', 'like']
an ['another', 'itself', 'athens', 'apoptosis', 'any', 'benzoic', 'amnesty', 'free']
when ['while', 'once', 'if', 'before', 'eventually', 'because', 'against', 'then']
also ['still', 'mainly', 'which', 'often', 'now', 'actually', 'sometimes', 'eventually']
are ['were', 'is', 'being', 'themselves', 'include', 'various', 'both', 'have']
be ['being', 'been', 'often', 'usually', 'is', 'have', 'generally', 'sometimes']
not ['never', 'still', 'if', 'usually', 'actually', 'without', 'nor', 'always']
use ['form', 'brain', 'existence', 'cause', 'value', 'share', 'using', 'provide']
there ['longer', 'indeed', 'therefore', 'true', 'fairly', 'neither', 'actually', 'still']
hi

Average loss: 7.3603510376: 100%|██████████| 200001/200001 [29:33<00:00, 112.80it/s]


Created the path: ./final_NLL_Updated_Params_1_ckpt


In [ ]:
#Learning rate to 0.5, Neg samples to 3, and Window size to 2: Params NLL
file_path = 'word2vec-word-analogies.txt'
model = torch.load('/content/NLL_Updated_Params_1_ckpt_negative_log_likelihood/200000.ckpt')
evaluate_word2vec(file_path, model)

Category                     Accuracy
capital-common-countries       0.000
capital-world                  0.000
currency                       0.000
city-in-state                  0.000
family                         0.000
gram1-adjective-to-adverb      0.000
gram2-opposite                 0.000
gram3-comparative              0.000
gram4-superlative              0.000
gram5-present-participle       0.000
gram6-nationality-adjective    0.000
gram7-past-tense               0.000
gram8-plural                   0.000
gram9-plural-verbs             0.000


In [ ]:
#Changed learning rate to 3, neg samples to 2, and window size to 4
run_training(
    model_type = 'negative_log_likelihood',  # specify the loss function used to train the model
    learning_rate = 3,  # the learning rate used in training the model
    num_neg_samples_per_center = 2,  # control for the number of negative samples per center word
    ckpt_model_path = './NLL_Updated_Params_2_ckpt',  # path to the model checkpoint
    final_model_path = './final_NLL_Updated_Params_2_ckpt',  # path to the trained model
    vocab_size = int(1e5),  # size of the vocabulary
    batch_size = 64,  # batch size: the number of (x,y) pairs used in training
    embedding_size = 128,  # size of the embedding vectors
    ckpt_step_size = 50000,  # number of steps after which model is saved as a checkpoint
    max_steps = 200001,  # the maximum number of training steps
    skip_window = 4
)



Created the path: ./NLL_Updated_Params_2_ckpt_negative_log_likelihood/
Data size 17005207
Top 10 most frequent tokens (+ UNK): [['UNK', 189230], ('the', 1061396), ('of', 593677), ('and', 416629), ('one', 411764), ('in', 372201), ('a', 325873), ('to', 316376), ('zero', 264975), ('nine', 250430)]
Sample data [5234, 3081, 12, 6, 195, 2, 3134, 46, 59, 156, 128, 742, 477, 10572, 134, 1, 27350, 2, 1, 103] ['anarchism', 'originated', 'as', 'a', 'term', 'of', 'abuse', 'first', 'used', 'against', 'early', 'working', 'class', 'radicals', 'including', 'the', 'diggers', 'of', 'the', 'english']
195 term  ->  5234 anarchism
195 term  ->  3081 originated
195 term  ->  12 as
195 term  ->  6 a
195 term  ->  2 of
195 term  ->  3134 abuse
195 term  ->  46 first
195 term  ->  59 used
2 of  ->  3081 originated
2 of  ->  12 as
Device: cuda:0


  0%|          | 0/200001 [00:00<?, ?it/s]

Printing closest words
between ['passacaglia', 'gauchos', 'rgensen', 'shortbread', 'second', 'ohr', 'unrolling', 'commemorated']


Average loss: 28.7542967796:   0%|          | 1/200001 [00:00<20:19:33,  2.73it/s]

see ['artefact', 'cassell', 'nda', 'windu', 'amphitrite', 'bulger', 'protestor', 'sympathized']
more ['aventinus', 'risch', 'earnest', 'stj', 'via', 'xidan', 'bellingshausen', 'stabat']
i ['fathers', 'discrediting', 'stomp', 'seinfeld', 'igoe', 'ludovicus', 'amount', 'mausolea']
people ['kra', 'standalone', 'whereabouts', 'maracas', 'furneaux', 'gulden', 'crvenkovski', 'oligarchical']
have ['krypt', 'muqawama', 'ghosting', 'wavetable', 'heterozygote', 'unite', 'kwame', 'avvocato']
system ['ktiv', 'pnc', 'trujillo', 'tatmadaw', 'geocentric', 'daphnia', 'alienation', 'barnyard']
a ['naturae', 'agone', 'contracts', 'snape', 'monomyth', 'snowiest', 'begum', 'pwnage']
they ['cua', 'barz', 'hes', 'wintel', 'kananaskis', 'nuv', 'lbf', 'fragile']
some ['parametric', 'hocking', 'metro', 'burgo', 'mwi', 'revoked', 'fertilizer', 'satellites']
the ['seac', 'outbursts', 'affricated', 'stern', 'mithridates', 'encapsulating', 'test', 'masculinity']
only ['oneself', 'surigao', 'bonanza', 'mup', 'artap

...

have ['hillary', 'asterion', 'beverley', 'magic', 'barbarism', 'opting', 'apocrypha', 'somewhere']
system ['su', 'mercantilism', 'dignify', 'gaulle', 'obligated', 'lendering', 'depictions', 'specialties']
a ['chicago', 'akm', 'heightened', 'bureaucratic', 'backlight', 'spoken', 'torino', 'kb']
they ['angels', 'yehudim', 'emulator', 'nergal', 'refering', 'waking', 'blumberg', 'kettering']
some ['erstwhile', 'spelunkers', 'isabeau', 'herzl', 'demeanor', 'vigilante', 'camillo', 'ology']
the ['gunners', 'fireside', 'riesling', 'jang', 'fads', 'colloidal', 'interested', 'producers']
only ['palus', 'dortch', 'karkar', 'auden', 'pukapuka', 'inf', 'metaphysical', 'inspirational']
but ['morin', 'transliterations', 'tiresome', 'copilot', 'sandiego', 'angulus', 'orthographic', 'durango']
its ['tractable', 'monaco', 'imperialist', 'garments', 'fansite', 'verlag', 'implicature', 'cargo']
all ['betrayals', 'gaulish', 'trophies', 'sparkle', 'pooh', 'nazis', 'clerks', 'radicalization']
this ['metanarr

Average loss: 3652.1670602379: 100%|█████████▉| 199988/200001 [29:28<00:00, 126.51it/s]

Printing closest words
between ['fatter', 'terms', 'isbn', 'step', 'levying', 'hito', 'newsom', 'minima']
see ['krofft', 'fro', 'spokesperson', 'po', 'necropolis', 'masks', 'kilmarnock', 'montevideo']
more ['reason', 'electrodes', 'bosporus', 'nom', 'continental', 'karoo', 'trend', 'apologists']
i ['bugfixes', 'angiotensin', 'dhea', 'morveau', 'guyton', 'quintet', 'warspite', 'montesquieu']
people ['didjeridoo', 'shiloah', 'seafoam', 'roe', 'econometric', 'margravate', 'killed', 'nations']
have ['differential', 'billed', 'carmel', 'banknote', 'patterned', 'mischa', 'pashto', 'hilda']
system ['colleagues', 'landslide', 'keri', 'assignment', 'mounting', 'interpret', 'margret', 'honorius']
a ['q', 'barnabas', 'beaked', 'orlando', 'lactamase', 'cocteau', 'chemnitz', 'genoese']
they ['gait', 'britton', 'mechanized', 'autism', 'spectators', 'routed', 'mccoy', 'stemmed']
some ['asrock', 'demogroup', 'privileges', 'skee', 'hydrodynamic', 'maildir', 'iulius', 'niedermayer']
the ['tim', 'marisa'

Average loss: 3652.1670602379: 100%|██████████| 200001/200001 [29:28<00:00, 113.06it/s]


Created the path: ./final_NLL_Updated_Params_2_ckpt


In [ ]:
#Learning rate to 3, Neg samples to 2, and Window size to 4: Params NLL
#Learing rate was too high our loss increased
file_path = 'word2vec-word-analogies.txt'
model = torch.load('/content/NLL_Updated_Params_2_ckpt_negative_log_likelihood/200000.ckpt')
evaluate_word2vec(file_path, model)

Category                     Accuracy
capital-common-countries       0.000
capital-world                  0.000
currency                       0.000
city-in-state                  0.000
family                         0.000
gram1-adjective-to-adverb      0.000
gram2-opposite                 0.000
gram3-comparative              0.000
gram4-superlative              0.000
gram5-present-participle       0.000
gram6-nationality-adjective    0.000
gram7-past-tense               0.000
gram8-plural                   0.000
gram9-plural-verbs             0.000


In [ ]:
#Changed learning rate to 0.5, neg samples to 3, and window size to 2
run_training(
    model_type = 'noise_contrastive_estimation',  # specify the loss function used to train the model
    learning_rate = 0.5,  # the learning rate used in training the model
    num_neg_samples_per_center = 3,  # control for the number of negative samples per center word
    ckpt_model_path = './SiNCE_Updated_Params_1_ckpt',  # path to the model checkpoint
    final_model_path = './final_SiNCE_Updated_Params_1_ckpt',  # path to the trained model
    vocab_size = int(1e5),  # size of the vocabulary
    batch_size = 64,  # batch size: the number of (x,y) pairs used in training
    embedding_size = 128,  # size of the embedding vectors
    ckpt_step_size = 50000,  # number of steps after which model is saved as a checkpoint
    max_steps = 200001,  # the maximum number of training steps
    skip_window = 2
)

Created the path: ./SiNCE_Updated_Params_1_ckpt_noise_contrastive_estimation/
Data size 17005207
Top 10 most frequent tokens (+ UNK): [['UNK', 189230], ('the', 1061396), ('of', 593677), ('and', 416629), ('one', 411764), ('in', 372201), ('a', 325873), ('to', 316376), ('zero', 264975), ('nine', 250430)]
Sample data [5234, 3081, 12, 6, 195, 2, 3134, 46, 59, 156, 128, 742, 477, 10572, 134, 1, 27350, 2, 1, 103] ['anarchism', 'originated', 'as', 'a', 'term', 'of', 'abuse', 'first', 'used', 'against', 'early', 'working', 'class', 'radicals', 'including', 'the', 'diggers', 'of', 'the', 'english']
12 as  ->  5234 anarchism
12 as  ->  3081 originated
12 as  ->  6 a
12 as  ->  195 term
6 a  ->  3081 originated
6 a  ->  12 as
6 a  ->  195 term
6 a  ->  2 of
195 term  ->  12 as
195 term  ->  6 a
Device: cuda:0


  0%|          | 0/200001 [00:00<?, ?it/s]

Printing closest words
over ['tsarevich', 'mspf', 'wittmann', 'waving', 'refa', 'falkland', 'cloud', 'tombaugh']
used ['policemen', 'prk', 'chp', 'entheogens', 'left', 'frisco', 'worrisome', 'michel']
a ['afternoons', 'vim', 'tritton', 'dachau', 'phonons', 'founding', 'zsef', 'unavoidable']


Average loss: 4.1591403484:   0%|          | 1/200001 [00:00<19:17:09,  2.88it/s]

as ['philm', 'boss', 'catalana', 'pki', 'avoiding', 'inviscid', 'lecterns', 'gyatso']
were ['crewman', 'esd', 'bonneville', 'conjectures', 'countrywide', 'deuteronomist', 'happier', 'surprises']
zero ['impulsively', 'tied', 'megagames', 'nacn', 'bca', 'asprin', 'gels', 'gehlen']
time ['devonian', 'divulged', 'cannibalization', 'strategy', 'foretells', 'bursts', 'udvar', 'uncle']
at ['sheepskin', 'osf', 'colon', 'rodman', 'rla', 'airhead', 'discussed', 'bale']
has ['jerky', 'millsaps', 'acquire', 'sepals', 'sari', 'unnerving', 'buttress', 'harmonica']
s ['inspiring', 'byzas', 'viatour', 'sparkling', 'witch', 'bextor', 'santamaria', 'sherbet']
which ['alonso', 'sacd', 'cybulski', 'uruguay', 'amphitheater', 'northwich', 'mikhoels', 'sixty']
more ['timoshenko', 'alimentary', 'incrementally', 'luki', 'thurgood', 'dehaene', 'lucrinus', 'after']
use ['slip', 'oeuvres', 'heterosexually', 'verres', 'pennyroyal', 'salicylates', 'qian', 'augusta']
from ['podge', 'franklin', 'magistracies', 'dhyan

...

time ['first', 'there', 'year', 'part', 'known', 'years', 'end', 'if']
at ['all', 'after', 'but', 'which', 'other', 'during', 'also', 'many']
has ['have', 'had', 'it', 'if', 'but', 'also', 'there', 'been']
s ['after', 'at', 'which', 'first', 'his', 'also', 'other', 'during']
which ['this', 'its', 'many', 'other', 'has', 'also', 'all', 'but']
more ['all', 'use', 'then', 'would', 'because', 'some', 'they', 'most']
use ['people', 'any', 'known', 'end', 'however', 'about', 'them', 'east']
from ['many', 'which', 'its', 'other', 'this', 'their', 'but', 'were']
known ['later', 'use', 'people', 'however', 'name', 'if', 'them', 'series']
there ['people', 'see', 'history', 'years', 'however', 'used', 'during', 'time']


Average loss: 1.2839365699: 100%|█████████▉| 199994/200001 [24:26<00:00, 155.76it/s]

Printing closest words
over ['year', 'years', 'if', 'east', 'all', 'people', 'there', 'use']
used ['there', 'people', 'world', 'since', 'later', 'see', 'then', 'use']
a ['this', 'which', 'or', 'an', 'its', 'as', 'is', 'his']
as ['which', 'but', 'many', 'this', 'also', 'is', 'are', 'has']
were ['being', 'many', 'these', 'has', 'they', 'very', 'been', 'government']
zero ['nine', 'eight', 'five', 'six', 'one', 'seven', 'three', 'four']
time ['first', 'there', 'year', 'part', 'known', 'years', 'end', 'history']
at ['all', 'after', 'which', 'but', 'during', 'other', 'many', 'also']
has ['have', 'had', 'it', 'if', 'but', 'also', 'there', 'been']
s ['after', 'at', 'which', 'his', 'first', 'also', 'other', 'during']
which ['this', 'its', 'many', 'other', 'has', 'all', 'also', 'but']
more ['all', 'use', 'then', 'would', 'because', 'some', 'most', 'they']
use ['people', 'any', 'known', 'end', 'however', 'east', 'about', 'each']
from ['which', 'many', 'its', 'this', 'other', 'their', 'but', 'were

Average loss: 1.2839365699: 100%|██████████| 200001/200001 [24:27<00:00, 136.33it/s]


Created the path: ./final_SiNCE_Updated_Params_1_ckpt


In [ ]:
#Learning rate to 0.5, Neg samples to 3, and Window size to 2
file_path = 'word2vec-word-analogies.txt'
model = torch.load('/content/SiNCE_Updated_Params_1_ckpt_noise_contrastive_estimation/200000.ckpt')
evaluate_word2vec(file_path, model)

Category                     Accuracy
capital-common-countries       0.000
capital-world                  0.000
currency                       0.000
city-in-state                  0.000
family                         0.000
gram1-adjective-to-adverb      0.000
gram2-opposite                 0.000
gram3-comparative              0.000
gram4-superlative              0.000
gram5-present-participle       0.000
gram6-nationality-adjective    0.000
gram7-past-tense               0.000
gram8-plural                   0.000
gram9-plural-verbs             0.000


In [ ]:
#Changed learning rate to 3, neg samples to 2, and window size to 4: Params SiNCE
run_training(
    model_type = 'noise_contrastive_estimation',  # specify the loss function used to train the model
    learning_rate = 3,  # the learning rate used in training the model
    num_neg_samples_per_center = 2,  # control for the number of negative samples per center word
    ckpt_model_path = './SiNCE_Updated_Params_2_ckpt',  # path to the model checkpoint
    final_model_path = './final_SiNCE_Updated_Params_2_ckpt',  # path to the trained model
    vocab_size = int(1e5),  # size of the vocabulary
    batch_size = 64,  # batch size: the number of (x,y) pairs used in training
    embedding_size = 128,  # size of the embedding vectors
    ckpt_step_size = 50000,  # number of steps after which model is saved as a checkpoint
    max_steps = 200001,  # the maximum number of training steps
    skip_window = 4
)

Created the path: ./SiNCE_Updated_Params_2_ckpt_noise_contrastive_estimation/
Data size 17005207
Top 10 most frequent tokens (+ UNK): [['UNK', 189230], ('the', 1061396), ('of', 593677), ('and', 416629), ('one', 411764), ('in', 372201), ('a', 325873), ('to', 316376), ('zero', 264975), ('nine', 250430)]
Sample data [5234, 3081, 12, 6, 195, 2, 3134, 46, 59, 156, 128, 742, 477, 10572, 134, 1, 27350, 2, 1, 103] ['anarchism', 'originated', 'as', 'a', 'term', 'of', 'abuse', 'first', 'used', 'against', 'early', 'working', 'class', 'radicals', 'including', 'the', 'diggers', 'of', 'the', 'english']
195 term  ->  5234 anarchism
195 term  ->  3081 originated
195 term  ->  12 as
195 term  ->  6 a
195 term  ->  2 of
195 term  ->  3134 abuse
195 term  ->  46 first
195 term  ->  59 used
2 of  ->  3081 originated
2 of  ->  12 as
Device: cuda:0


  0%|          | 0/200001 [00:00<?, ?it/s]

Printing closest words
system ['exiguus', 'hanya', 'falkenberg', 'melba', 'vulnerable', 'unsatisfied', 'apostates', 'herlihy']
american ['bust', 'vesta', 'nadu', 'contour', 'depicting', 'flirts', 'tzedakah', 'hutongs']
during ['muddied', 'matilde', 'celera', 'yakovlev', 'albrechtsberger', 'iwc', 'bleaching', 'animator']
he ['esera', 'sheila', 'windhover', 'alka', 'sponges', 'marthasville', 'laughing', 'wurst']
history ['emmerich', 'nobita', 'featurettes', 'adt', 'snooker', 'shigeru', 'calvins', 'amalric']
two ['silting', 'liga', 'andie', 'steamy', 'grout', 'milli', 'assonance', 'interred']
states ['pads', 'aj', 'gling', 'spayed', 'itil', 'valentino', 'jemaah', 'input']
b ['baruch', 'logon', 'eskimos', 'crystals', 'lhd', 'rajiv', 'malar', 'mahayana']
by ['detaining', 'rises', 'abderus', 'siculus', 'structurally', 'expansion', 'fling', 'liaisons']
his ['thermians', 'fullers', 'foraminifera', 'daniell', 'canossa', 'marzen', 'rebuff', 'morals']
from ['underdog', 'provincehood', 'skimmed', 

Average loss: 1.90561382:   0%|          | 999/200001 [00:07<23:41, 139.95it/s]  

Printing closest words
system ['of', 'a', 'often', 'anarchism', 'the', 'have', 'as', 'most']
american ['autism', 'such', 'of', 'groups', 'anarchist', 'the', 'years', 'as']
during ['however', 'was', 'anarchism', 'with', 'post', 'on', 'has', 'in']
he ['and', 'of', 'in', 'the', 'anarchists', 'as', 'is', 'a']
history

...

he ['him', 'who', 'she', 'when', 'death', 'had', 'god', 'them']
history ['centre', 'mountains', 'explained', 'introduced', 'cloister', 'maryland', 'dynasties', 'associates']
two ['one', 'five', 'km', 'ca', 'births', 'january', 'nfl', 'zero']
states ['united', 'finished', 'congress', 'constitution', 'arizona', 'drive', 'beginning', 'china']
b ['singer', 'writer', 'actress', 'politician', 'composer', 'musician', 'd', 'swedish']
by ['informal', 'pottery', 'arabs', 'grass', 'threatens', 'pocket', 'professionals', 'atlas']
his ['him', 'her', 'wife', 'father', 'son', 'king', 'daughter', 'muhammad']
from ['european', 'ill', 'benz', 'dark', 'archaeological', 'inventory', 'agnes', 'via']
not ['should', 'existence', 'we', 'did', 'you', 'might', 'but', 'those']
be ['should', 'can', 'might', 'would', 'could', 'if', 'reason', 'money']
into ['arm', 'dense', 'load', 'describing', 'arms', 'changing', 'forward', 'foundation']
used ['wide', 'terms', 'antibiotics', 'similar', 'variety', 'remarkably', 'fu

Average loss: 0.671694501: 100%|█████████▉| 199991/200001 [23:40<00:00, 151.19it/s] 

Printing closest words
system ['selection', 'produced', 'decay', 'systems', 'albatrosses', 'process', 'rings', 'sound']
american ['actor', 'musician', 'canadian', 'singer', 'politician', 'swedish', 'frank', 'actress']
during ['movies', 'got', 'throne', 'reported', 'invaded', 'leone', 'reached', 'compared']
he ['him', 'who', 'she', 'when', 'them', 'death', 'did', 'had']
history ['centre', 'mountains', 'introduced', 'explained', 'dynasties', 'cloister', 'associates', 'listing']
two ['nfl', 'five', 'vol', 'november', 'km', 'january', 'one', 'ca']
states ['united', 'finished', 'congress', 'arizona', 'constitution', 'china', 'drive', 'tennessee']
b ['singer', 'writer', 'actress', 'politician', 'composer', 'musician', 'swedish', 'footballer']
by ['pottery', 'informal', 'arabs', 'ministry', 'grass', 'tourists', 'quality', 'sank']
his ['him', 'her', 'wife', 'king', 'father', 'life', 'who', 'muhammad']
from ['european', 'ill', 'examiner', 'pompeius', 'dark', 'schleswig', 'benz', 'inventory']
no

Average loss: 0.671694501: 100%|██████████| 200001/200001 [23:41<00:00, 140.72it/s]


Created the path: ./final_SiNCE_Updated_Params_2_ckpt


In [ ]:
#Learning rate to 3, Neg samples to 2, and Window size to 4: Params NLL
file_path = 'word2vec-word-analogies.txt'
model = torch.load('/content/SiNCE_Updated_Params_2_ckpt_noise_contrastive_estimation/200000.ckpt')
evaluate_word2vec(file_path, model)

Category                     Accuracy
capital-common-countries       0.000
capital-world                  0.000
currency                       0.000
city-in-state                  0.000
family                         0.000
gram1-adjective-to-adverb      0.000
gram2-opposite                 0.000
gram3-comparative              0.000
gram4-superlative              0.000
gram5-present-participle       0.000
gram6-nationality-adjective    0.000
gram7-past-tense               0.000
gram8-plural                   0.000
gram9-plural-verbs             0.000


**5.1.1** Of the many combinations possible, what are the hyperparameters selected for this experiment (provide this information for every run)?

The orginal hyperparameters were the best parameters I was able to find for both the NLL and SiNCE. Even though they both had poor accuracy you could see while they were training that they were begining to learn the analogies and would maybe would be able to better with a larger step size.

**5.1.2** Why did you select these values?

I attemped to choose hyperparameters out of the ones given that would have the largest effect on my models since they were producing poor reesults. I tried things like lowering and raising leraning rate, window size, and negative samples but couldn't seem to find an increase. However it should be noted that messing with the hyperparameters does provide insight into the importance of some of them for example when turning the learning rate too high for the model you can see that we actually step over the optimization and instead gain loss throughout training.

### 5.2 Analogies and Relations

Words can be "similar" in many ways, and the various relational categories you saw in the word analogy test file have hopefully convinced you even further!

But, the results show that for various hyperparameter combinations, the category-specific results vary. Let's look at the various grammatical relations first:

**5.2.1** How many total relation categories are there in the `word2vec-word-analogies.txt` test file? Of these, please list the ones based on English grammar.

There are 14 total relation categories in the `word2vec-word-analogies.txt` test file. The ones based on English grammar are gram1-adjective-to-adverb, gram2-opposite, gram3-comparative, gram4-superlative, gram5-present-participle, gram6-nationality-adjective, gram7-past-tense, gram8-plural, gram9-plural-verbs so there are 9 of these.

**5.2.2** Among the grammatical relations you just listed, which model performs the best (on average)? Which model yielded the best in any specific relation category (e.g., $M$ may have performed exceptionally well in `gram9-plural-verbs`, even if it had horrible accuracy everywhere else)?

My NLL model yeilded the best results overall in all since all others had 0 percent accuracry. The NLL model was able to find some gram3-comparative and gram9-plural-verbs with very low accuracy but being able to complete any at all is a success.

**5.2.3** What do you think are the reasons behind your previous answer? That is, why do you think *that model* was better in *that/those category/ies*?

Since I only had one model that yielded good results I will make an informed descision on to why this would happen even though I didn't expereience it. I belive this is due to the fact that some semantical relationships can be understood better with certain hyperparameter. Some analogies may be able to only be analyzed when using a larger window size to capture more context words for each target word.

**5.2.4** Describe how you can design an experiment to prove/disprove your hypotheses in the above answer.

The way to test this would be to see if different hyperparameters espcesially ones like window size and embedding size have any effect on different categories. Some analogis or relationships may be more simple and not require a large window size or embeddding size to capture them and actually may do worse with a larger hyperparmeters since they are trying to capture more nuanced realationships.


### 5.3 Dense embeddings in classification

Do you think the Word2Vec embeddings (or, for that matter, the pretrained GloVe embeddings) can be used in binary classification? For example, in binary sentiment classification of movie reviews? If yes, write a short paragraph describing precisely how you would do this. If no, provide a technical justification why this is not possible.

Your answer should address technical aspects of binary classification such as what the instance representation will be (the instance is a document, not a word, so this needs elaboration), how the classifier's decision makes sense (e.g., is it probabilistic? if so, is the probability score logical?)

Yes I believe that the Word2Vec embeddings (or the pretrained GloVe embeddings) can be used in a binary classification task. Using the example given of binary sentiemnt classification of movie reviews we can first take a review and do the same thing that we did above which was processing our text into a vocabulary, dicts and batches. Then through this we can train our model on these batches to get the embeddings of each word. Once we have the embedings of each word we can combined all these word embedding to get a single document embedding. Therefore, our instance is the combined vector embeddings of all the words in the document. The one issue that might arise is different length documents would have different lengths/heights depending on word count so we would need a way to normalize each document embedding either maybe by transforming them all into a fixed length embedding for all of them. Finally with this document embedding we are able to use something like a logistic regression to determine whether this embedding is postive or negative since we will have trained our logistic regression on document embeddings in which we alredy had the classification for. This would be probabilisitic since logistic regression is a probabilistic classifier meaning that it will give us the probabily that we are in the positive or negative.

Suppose you are tasked with using Word2Vec (or GloVe) embeddings for named entity recognition. Describe in 1-2 paragraphs how you could have used them in your previous assignment. Illustrate with at least one instance from the training or test set in the previous assignment. Your answer should explain how an instance will be represented, and how/why the final classification actually makes sense. Your design shouldn't be an act of just blindly throwing embeddings at a problem and hoping it will solve something! We expect that your answer will demonstrate a good understanding of NER as well as dense embeddings.

In our previous assingment on NER we could haved used Word2Vec (or GloVe) embedding to help classify the tokens. In the NER assingment we were classifying our token based on features that we choose such as the POS tags and tags of nearby words. With the help of these embeddings we are able to have an additonal and more semantically rich features that our classifier can then use to more accurately predict the correct NER for the token. We would do this by getting our vector embeddings from the text like how we did in this assingment. We know similar embeddings typically are stored near each other so we could also infer that similar embeddings also have similar NER tags since similar embeddings should be synonyms. On top of using these embeddings for features alone (which as stated blindly throwing embedding  may not be the best option)  we can also use near by words that have already been classified in our training set to help classify the current token. For example, "Harry Belafonte, the popular American singer, actor, and civili rights activist, was born Harold George Bellanfanti Jr. in 1927, at Lying-in Hospital in Harlem, New York." could be an example of a text where we are trying to find the NER of tokens in the text. Lets say we are trying to just classify one token in that text with its proper NER tag like 'Harlem'. After training and learning the embedding of all the words/tokens in the text we get an embedding for 'Harlem'. We then are able to see the closest words to 'harlem'. Lets say the 3 closest words are 'manhattan', 'bronx', and 'city' of which we may already have learned some of their NER tags from our training data. We can then use the fact that these similar words are all locations to create a new feature like nearbyWords embeddings which will contain a onehot encoding of the 3 closest words predicted NER tags which will drastically help in the classification of our current NER tag but may have the disadvantage of efficency. It would be important to test with and without embedding to see if the extra time it takes to get the embeddings out weighs the increase in accuracy we achieve.